In [1]:
import array, numpy as np

# array.array('B') are unsigned char in C
# Just enough to contain any ascii charactor

def vernam_operation(src_array,keystream):
    key_len = len(keystream)
    return array.array('B',[np.bitwise_xor(v,keystream[i % key_len]) for i,v in enumerate(src_array)])

# Convert bytearray to sth like FF FA 01 BB
def bytearray_to_readable(btarr):
    return " ".join([hex(b)[2:].upper().zfill(2) for b in btarr])

def bytearray_to_string(arr):
    return "".join([chr(v) for v in arr])
def string_to_bytearray(chars):
    assert chars.isascii(), "String must be ASCII charactors."
    return array.array('B',[ord(v) for v in chars])

# All parameters involving plaintexts or ciphertexts assume bytearray
class Vernam:
    def __init__(self,keystream):
        self.keystream = keystream
    @staticmethod
    def keygen(length):
        gen = np.random.default_rng()
        keystream = array.array('B')
        bitlen = 2 ** (keystream.itemsize * 8)
        for _ in range(length):
            keystream.append(gen.integers(0,bitlen))
        return keystream
    def operation(self,a):
        return vernam_operation(a,self.keystream)
    # Reverse of XOR is also XOR
    # Creating shorcuts for readibility
    encrypt = operation
    decrypt = operation

In [2]:
key = Vernam.keygen(10) # Generate a fixed-length key, Less secure than OTP
print(bytearray_to_readable(key))

E1 0A CB DF 71 32 1A 48 CF 8B


In [3]:
ve = Vernam(key)
m = "We are discovered, safe yourself."
m_btarr = string_to_bytearray(m)
c = ve.encrypt(m_btarr)
rtn_m_btarr = ve.decrypt(c)
rtn_m = bytearray_to_string(rtn_m_btarr)

print(m,bytearray_to_readable(m_btarr))
print(bytearray_to_readable(c))
print(rtn_m,bytearray_to_readable(rtn_m_btarr))

We are discovered, safe yourself. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E
B6 6F EB BE 03 57 3A 2C A6 F8 82 65 BD BA 03 57 7E 64 EF F8 80 6C AE FF 08 5D 6F 3A BC EE 8D 6C E5
We are discovered, safe yourself. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E


## One-time Pad (OTP)

In [4]:
def OTP_encrypt(m):
    key = Vernam.keygen(len(m))
    return Vernam(key).encrypt(m), key

def OTP_decrypt(c,key):
    return Vernam(key).decrypt(c)

for i in range(5):
    print("Run {}".format(i + 1))
    m = "We are discovered, safe yourself."
    m_btarr = string_to_bytearray(m)
    c, key = OTP_encrypt(m_btarr)
    rtn_m_btarr = OTP_decrypt(c, key)
    rtn_m = bytearray_to_string(rtn_m_btarr)

    print(bytearray_to_readable(key))
    print(m,bytearray_to_readable(m_btarr))
    print(bytearray_to_readable(c))
    print(rtn_m,bytearray_to_readable(rtn_m_btarr))

Run 1
D7 0A 8C D1 5C 5F 2C 08 45 FC 95 94 B7 CF E5 1C 1D A1 19 D9 DF 84 19 0A 89 7B DD 55 7F A6 49 43 60
We are discovered, safe yourself. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E
80 6F AC B0 2E 3A 0C 6C 2C 8F F6 FB C1 AA 97 79 79 8D 39 AA BE E2 7C 2A F0 14 A8 27 0C C3 25 25 4E
We are discovered, safe yourself. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E
Run 2
98 EB 5F 30 7D 12 0A 43 9B 0A 93 53 FA 0C E6 27 5D 87 6F 86 21 3A D2 40 3F 54 7E BA 80 9B F5 02 F8
We are discovered, safe yourself. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E
CF 8E 7F 51 0F 77 2A 27 F2 79 F0 3C 8C 69 94 42 39 AB 4F F5 40 5C B7 60 46 3B 0B C8 F3 FE 99 64 D6
We are discovered, safe yourself. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E
Run 3
37 F3 18 26 7F 17 01 C6 79 49 19 3A CE CC 7D 8A 4B E7 

## Non-ASCII Exception

In [5]:
string_to_bytearray("Happy 測試")

AssertionError: String must be ASCII charactors.